In [ ]:
import numpy as np
import pandas as pd
from dputils.scrape import Scraper, Tag, Browser
from dputils.scrape import BeautifulSoup, requests
import time

In [26]:
BASE_URL = 'https://www.imdb.com'
IMDB_URL = f'{BASE_URL}/chart/moviemeter'

In [27]:
sc = Scraper(IMDB_URL)

In [35]:
target = Tag('div', cls='sc-9bf20411-3 khhOiz ipc-page-grid__item ipc-page-grid__item--span-2')
items =Tag('li')
img = Tag('img', output='src')
rank = Tag(cls='sc-94da5b1b-0 kaCAvv meter-const-ranking sc-14dd939d-4 iuctsm cli-meter-title-header')
title = Tag(cls='ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-14dd939d-7 fjdYTb cli-title')
metadata = Tag(cls='sc-14dd939d-5 cPiUKY cli-title-metadata')
rating = Tag('span', cls='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')
popular_movies = sc.get_all(
    target,
    items, 
    img=img, 
    rank=rank, 
    title=title, 
    metadata=metadata, 
    rating=rating
)
popular_movies

[{'img': 'https://m.media-amazon.com/images/M/MV5BMDBmYTZjNjUtN2M1MS00MTQ2LTk2ODgtNzc2M2QyZGE5NTVjXkEyXkFqcGdeQXVyNzAwMjU2MTY@._V1_QL75_UX140_CR0,0,140,207_.jpg',
  'rank': '1 ()',
  'title': 'Oppenheimer',
  'metadata': '20233hR',
  'rating': '8.7'},
 {'img': 'https://m.media-amazon.com/images/M/MV5BNjU3N2QxNzYtMjk1NC00MTc4LTk1NTQtMmUxNTljM2I0NDA5XkEyXkFqcGdeQXVyODE5NzE3OTE@._V1_QL75_UX140_CR0,0,140,207_.jpg',
  'rank': '2 ()',
  'title': 'Barbie',
  'metadata': '20231h 54mUA',
  'rating': '7.4'},
 {'img': 'https://m.media-amazon.com/images/M/MV5BYzFiZjc1YzctMDY3Zi00NGE5LTlmNWEtN2Q3OWFjYjY1NGM2XkEyXkFqcGdeQXVyMTUyMTUzNjQ0._V1_QL75_UY207_CR1,0,140,207_.jpg',
  'rank': '3 ()',
  'title': 'Mission: Impossible - Dead Reckoning Part One',
  'metadata': '20232h 43mUA',
  'rating': '8.0'},
 {'img': 'https://m.media-amazon.com/images/M/MV5BNTAwNWQ1ZjQtZDFmMC00YjNlLWFlNjItMjJlYzZkMjJlNzkyXkEyXkFqcGdeQXVyNzc0MTgzMzU@._V1_QL75_UX140_CR0,0,140,207_.jpg',
  'rank': '4 ()',
  'title': 'Sound of Fre

In [38]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, Float
from sqlalchemy.orm import mapper, sessionmaker, declarative_base
import pandas as pd


In [39]:
Base = declarative_base()
class Movie(Base):
    __tablename__ = 'movies'
    id = Column(Integer, primary_key=True)
    title = Column(String(255))
    img_url = Column(Integer)
    rating = Column(Float)
    rank = Column(Integer)
    year = Column(String(255))
    age_rating = Column(String(255))
    runtime = Column(String(255))

    def __repr__(self):
        return self.title

    def __str__(self):
        return self.title


engine = create_engine('sqlite:///movies.db')
Base.metadata.create_all(engine)

In [54]:
db = sessionmaker(bind=engine)()
age_rating_list = ['R', 'PG-13','TV-MA','PG','UA']
for idx,record in enumerate(popular_movies):
    metadata= record['metadata']
    year = metadata[:4]
    for rl in age_rating_list:
        if rl in metadata:
            age_rating = rl
            break
    else:
        age_rating = 'U'
    title = record['title'].strip().capitalize()
    runtime = metadata.replace(year,'').replace(age_rating,'').strip()
    rating = record['rating']
    if rating and '.' in rating:
        rating = float(rating)
    else:
        try:rating = int(rating)
        except:rating = 0
    print(metadata, '--->',year, age_rating, title, runtime, rating)
    m = Movie(
        title=title,
        img_url=record['img'],
        rating=rating,
        rank=record['rank'],
        year=year,
        age_rating=age_rating,
        runtime=runtime)
    db.add(m)
db.commit()
    


20233hR ---> 2023 R Oppenheimer 3h 8.7
20231h 54mUA ---> 2023 UA Barbie 1h 54m 7.4
20232h 43mUA ---> 2023 UA Mission: impossible - dead reckoning part one 2h 43m 8.0
20232h 11mPG-13 ---> 2023 PG-13 Sound of freedom 2h 11m 7.9
20232h 2mR ---> 2023 R They cloned tyrone 2h 2m 6.7
20232h 24mUA ---> 2023 UA The flash 2h 24m 6.9
20232h 1m ---> 2023 U Exorcist 2h 1m 0
20221h 35mR ---> 2022 R Talk to me 1h 35m 7.5
20232h 3mPG-13 ---> 2023 PG-13 The haunted mansion 2h 3m 6.3
20232h 7mUA ---> 2023 UA Transformers: rise of the beasts 2h 7m 6.1
20232h 15mU ---> 2023 U The little mermaid 2h 15m 7.2
20232h 34mUA ---> 2023 UA Indiana jones and the dial of destiny 2h 34m 6.8
20223h 9m18 ---> 2022 U Babylon 3h 9m18 7.2
20231h 45mPG-13 ---> 2023 PG-13 Asteroid city 1h 45m 6.8
20142h 49mUA ---> 2014 UA Interstellar 2h 49m 8.7
20231h 35mA ---> 2023 U The out-laws 1h 35mA 5.4
20232h 30mUA ---> 2023 UA Guardians of the galaxy vol. 3 2h 30m 8.1
20232h 20mU ---> 2023 U Spider-man: across the spider-verse 2h 2

In [56]:
pd.read_sql('select * from movies', engine, index_col='id')

,title,img_url,rating,rank,year,age_rating,runtime
id,,,,,,,
1,Oppenheimer,https://m.media-amazon.com/images/M/MV5BMDBmYT...,8.7,1 (),2023,R,3h
2,Barbie,https://m.media-amazon.com/images/M/MV5BNjU3N2...,7.4,2 (),2023,UA,1h 54m
3,Mission: impossible - dead reckoning part one,https://m.media-amazon.com/images/M/MV5BYzFiZj...,8.0,3 (),2023,UA,2h 43m
4,Sound of freedom,https://m.media-amazon.com/images/M/MV5BNTAwNW...,7.9,4 (),2023,PG-13,2h 11m
5,They cloned tyrone,https://m.media-amazon.com/images/M/MV5BZGM2YT...,6.7,5 (19),2023,R,2h 2m
...,...,...,...,...,...,...,...
96,Nefarious,https://m.media-amazon.com/images/M/MV5BNmY4NT...,6.4,96 (18),2023,R,1h 37m
97,Midsommar,https://m.media-amazon.com/images/M/MV5BMzQxNz...,7.1,97 (12),2019,U,2h 28mA
98,Mission: impossible - ghost protocol,https://m.media-amazon.com/images/M/MV5BMTY4MT...,7.4,98 (60),2011,UA,2h 12m
